# Global Lesioning Analysis

Note: The first time this is run (i.e., with an empty cache), it will
take 2-3 hours from beginning to end.

In [1]:
import textwrap
import os
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from prep_data import \
    mazes,\
    model_preds,\
    get_exp1_at,\
    get_exp2_at,\
    get_exp3_at__exp3_mt,\
    get_exp4a_ht,\
    get_exp4b_ht
from analysisutils import short_predictor_names
import analysisutils

from vgc_project.r import create_R_model_interface, ImmutableDataFrame
joblib_cache_location = "./_analysiscache"
rmods = create_R_model_interface(joblib_cache_location=joblib_cache_location)


In [2]:
predictors = [
    'vgc_weight_Z',
    'log_traj_based_hitcount_Z',
    'graph_based_hitcount_Z',
    'bottleneck_dist_Z',
    'sr_occ_Z',
    'nav_mindist_Z',
    'nav_mindist_timestep_Z',
    'optpolicy_dist_Z',
    'goal_dist_Z',
    'start_dist_Z',
    'walls_dist_Z',
    'center_dist_Z',
]
exp_analysis_predictors = dict(
    exp1=[p for p in predictors if "optpolicy_" not in p],
    exp2_early=[p for p in predictors if "nav_mindist" not in p],
    exp2_full=[p for p in predictors if "optpolicy_" not in p],
    exp3=[p for p in predictors if "optpolicy_" not in p],
    exp4=[p for p in predictors if "nav_mindist" not in p],
)
    


In [3]:
exp1_at = get_exp1_at()
exp1_at_im = ImmutableDataFrame(exp1_at)
exp1_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp1']),
    data=exp1_at_im
)
exp1_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 1 Accuracy",
    data=exp1_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp1'],
    rmods=rmods
)

exp2_at = get_exp2_at()
exp2_early_at_im = ImmutableDataFrame(exp2_at[exp2_at['earlyterm'] == 'earlyterm'])
exp2_full_at_im = ImmutableDataFrame(exp2_at[exp2_at['earlyterm'] == 'full'])
exp2_at_im = ImmutableDataFrame(exp2_at)
exp2_full_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp2_full']),
    data=exp2_full_at_im
)
exp2_early_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp2_early']),
    data=exp2_early_at_im
)
exp2_early_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 2 Accuracy Early",
    data=exp2_early_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp2_early'],
    rmods=rmods
)
exp2_full_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 2 Accuracy Full",
    data=exp2_full_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp2_full'],
    rmods=rmods
)


exp3_at, exp3_mt = get_exp3_at__exp3_mt()
exp3_at_im = ImmutableDataFrame(exp3_at)
exp3_mt_im = ImmutableDataFrame(exp3_mt)
exp3_correct_global_model = rmods.lmer(
    formula="correct ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp3']),
    data=exp3_mt_im
)
exp3_conf_global_model = rmods.lmer(
    formula="conf_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp3']),
    data=exp3_mt_im
)
exp3_attn_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp3']),
    data=exp3_at_im
)
exp3_correct_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 3 Accuracy",
    data=exp3_mt_im,
    dv="correct",
    model_func="glmer",
    random_effects="(1 | sessionId)",
    predictors=exp_analysis_predictors['exp3'],
    rmods=rmods
)
exp3_conf_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 3 Confidence",
    data=exp3_mt_im,
    dv="conf_N",
    model_func="lmer",
    random_effects="(1 | sessionId)",
    predictors=exp_analysis_predictors['exp3'],
    rmods=rmods
)
exp3_attn_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 3 Awareness",
    data=exp3_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp3'],
    rmods=rmods
)


exp4a_ht = get_exp4a_ht()
exp4b_ht = get_exp4b_ht()
exp4a_ht_im = ImmutableDataFrame(exp4a_ht)
exp4b_ht_im = ImmutableDataFrame(exp4b_ht)
exp4a_hov_global_model = rmods.glmer(
    formula="hovered ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp4']),
    data=exp4a_ht_im
)
exp4a_dur_global_model = rmods.lmer(
    formula="log_hoverduration ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp4']),
    data=exp4a_ht_im
)
exp4b_hov_global_model = rmods.glmer(
    formula="hovered ~ (1 | sessionId) + (1 | grid) +"+"+".join(exp_analysis_predictors['exp4']),
    data=exp4b_ht_im
)
exp4b_dur_global_model = rmods.lmer(
    formula="log_hoverduration ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp4']),
    data=exp4b_ht_im
)
exp4a_hov_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4a Hover",
    data=exp4a_ht_im,
    dv="hovered",
    model_func="glmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)
exp4a_dur_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4a Hover Duration",
    data=exp4a_ht_im,
    dv="log_hoverduration",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)
exp4b_hov_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4b Hover",
    data=exp4b_ht_im,
    dv="hovered",
    model_func="glmer",
    random_effects="(1 | sessionId)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)
exp4b_dur_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4b Hover Duration",
    data=exp4b_ht_im,
    dv="log_hoverduration",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)

Loading Experiment 1 Attention Trials


100%|██████████| 11/11 [00:02<00:00,  3.84it/s]


## Tables with all estimated coefficients

In [4]:
def format_coefficients(mod):
    return pd.DataFrame(mod['coefficients']).\
        set_index('effect').\
        rename(columns={"estimate": r"$\beta$", "se": "SE"})
all_global_models = [
    dict(model=exp1_global_model, name=("Initial Exp", "Awareness")),
    dict(model=exp2_full_global_model, name=("Up-front Exp", "Awareness (F)")),
    dict(model=exp2_early_global_model, name=("Up-front Exp","Awareness (E)")),
    dict(model=exp3_correct_global_model, name=("Critical Mazes Exp", "Accuracy")),
    dict(model=exp3_conf_global_model, name=("Critical Mazes Exp", "Confidence")),
    dict(model=exp3_attn_global_model, name=("Critical Mazes Exp", "Awareness")),
    dict(model=exp4a_hov_global_model, name=("Process-Tracing Exp (Initial Mazes)", "Hovering")),
    dict(model=exp4a_dur_global_model, name=("Process-Tracing Exp (Initial Mazes)", "Duration")),
    dict(model=exp4b_hov_global_model, name=("Process-Tracing Exp (Critical Mazes)", "Hovering")),
    dict(model=exp4b_dur_global_model, name=("Process-Tracing Exp (Critical Mazes)", "Duration")),
]
coeff_table = pd.concat(
    [format_coefficients(mod['model']) for mod in all_global_models],
    axis=1,
    keys=[mod['name'] for mod in all_global_models]
)
coeff_table = coeff_table.reindex(["(Intercept)"]+predictors)
coeff_table.index = ["Intercept" if "Intercept" in i else short_predictor_names[i.replace("_Z", "")] for i in coeff_table.index.values]

In [5]:
coeff_table1_3 = coeff_table[["Initial Exp", "Up-front Exp", "Critical Mazes Exp"]]
coeff_string1_3 = coeff_table1_3.round(3).to_latex(
    na_rep="---", 
    escape=False,
    multicolumn_format='c',
    column_format="lrrrrrrrrrrrr",
    col_space=1,
)
coeff_string1_3 = coeff_string1_3.split('\n')

coeff_string1_3.insert(
    4, r"\cmidrule(lr){2-3} \cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}"
)
coeff_string1_3.insert(
    3, r"\cmidrule(lr){2-3} \cmidrule(lr){5-6} \cmidrule(lr){9-12}"
)
coeff_string1_3 = "\n".join(coeff_string1_3)
with open("results/coeff_table1_3.tex", "w") as file:
    file.write(
        textwrap.dedent(r"""
        \documentclass{standalone}
        \usepackage{booktabs}
        \usepackage{tabularx}
        \renewcommand{\familydefault}{\sfdefault}
        \begin{document}
        \fontsize{7}{8.4}
        \selectfont
        """) + \
        coeff_string1_3 + \
        textwrap.dedent(r"""
        \end{document}
        """)
    )

# Create pdf to combine in vector image editor
process = subprocess.Popen("pdflatex -output-directory results results/coeff_table1_3.tex".split())
output, error = process.communicate()
print(output, error)
coeff_table1_3

None None


Initial Exp            Up-front Exp                          \
                Awareness           Awareness (F)           Awareness (E)   
                  $\beta$        SE       $\beta$        SE       $\beta$   
Intercept        0.519297  0.016824      0.321033  0.018546      0.485064   
VGC              0.066759  0.003306      0.071472  0.004951      0.057174   
Traj HS         -0.038647  0.004984     -0.038230  0.007490     -0.021190   
Graph HS         0.067495  0.004354      0.054287  0.006571      0.043749   
Bottleneck      -0.032442  0.003355     -0.049435  0.005041     -0.057240   
SR Overlap      -0.036193  0.003113     -0.041163  0.004629     -0.040290   
Nav Dist        -0.107210  0.003508     -0.262859  0.012330           NaN   
Nav Dist Step    0.023727  0.003772     -0.023839  0.003464           NaN   
Opt Dist              NaN       NaN           NaN       NaN     -0.077121   
Goal Dist       -0.025168  0.005017     -0.021389  0.007117     -0.028658   
Start Dist      -0.028196  0.004693      0.009882  0.006936     -0.027109   
Wall Dist       -0.178961  0.024813     -0.094277  0.038436     -0.149365   
Center Dist      0.175427  0.025269      0.118027  0.039160      0.167496   

                        Critical Mazes Exp                                 \
                                  Accuracy           Confidence             
                     SE            $\beta$        SE    $\beta$        SE   
Intercept      0.016257           0.562127  0.074070   0.693911  0.067005   
VGC            0.005025           0.176394  0.019276   0.178484  0.012635   
Traj HS        0.007739          -0.070388  0.031480  -0.125965  0.026279   
Graph HS       0.006656           0.068942  0.035449   0.037359  0.026476   
Bottleneck     0.005187          -0.008517  0.014314  -0.048741  0.010540   
SR Overlap     0.004804          -0.143190  0.053221  -0.243987  0.045824   
Nav Dist            NaN          -0.061648  0.011865  -0.028905  0.007106   
Nav Dist Step       NaN          -0.008629  0.010535  -0.007492  0.006304   
Opt Dist       0.005543                NaN       NaN        NaN       NaN   
Goal Dist      0.007271           0.182603  0.082685  -0.049838  0.065309   
Start Dist     0.007032           0.206274  0.078388   0.004768  0.061196   
Wall Dist      0.037882           0.150567  0.084288   0.058645  0.052694   
Center Dist    0.038557          -0.189716  0.076242  -0.143459  0.048846   

                                   
              Awareness            
                $\beta$        SE  
Intercept      0.353754  0.073167  
VGC            0.266337  0.013686  
Traj HS       -0.208156  0.028335  
Graph HS       0.146715  0.028482  
Bottleneck    -0.084383  0.011354  
SR Overlap    -0.166429  0.049731  
Nav Dist      -0.099877  0.007485  
Nav Dist Step -0.046559  0.006700  
Opt Dist            NaN       NaN  
Goal Dist      0.278338  0.070875  
Start Dist     0.299972  0.066357  
Wall Dist      0.255132  0.056986  
Center Dist   -0.279569  0.052797

In [6]:
coeff_table4 = coeff_table[["Process-Tracing Exp (Initial Mazes)", "Process-Tracing Exp (Critical Mazes)"]].dropna(axis=0)
coeff_string4 = coeff_table4.round(3).to_latex(
        na_rep="---", 
        escape=False,
        multicolumn_format='c',
        column_format="lrrrrrrrrrrrr",
        col_space=1,
    )
coeff_string4 = coeff_string4.split('\n')

coeff_string4.insert(
    4, r"\cmidrule(lr){2-3} \cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9}"
)
coeff_string4.insert(
    3, r"\cmidrule(lr){2-5} \cmidrule(lr){6-9}"
)
coeff_string4 = "\n".join(coeff_string4)
with open("results/coeff_table4.tex", "w") as file:
    file.write(
        textwrap.dedent(r"""
        \documentclass{standalone}
        \usepackage{booktabs}
        \usepackage{tabularx}
        \renewcommand{\familydefault}{\sfdefault}
        \begin{document}
        \fontsize{7}{8.4}
        \selectfont
        """) + \
        coeff_string4 + \
        textwrap.dedent(r"""
        \end{document}
        """)
    )
    
# Create pdf to combine in vector image editor
process = subprocess.Popen("pdflatex -output-directory results results/coeff_table4.tex".split())
output, error = process.communicate()
print(output, error)
coeff_table4

None None


Process-Tracing Exp (Initial Mazes)                                \
                                       Hovering            Duration             
                                        $\beta$        SE   $\beta$        SE   
Intercept                              0.627561  0.145597  6.494925  0.079683   
VGC                                    0.487555  0.028845  0.153561  0.016163   
Traj HS                                0.179154  0.046523 -0.035189  0.027805   
Graph HS                               0.038627  0.040333  0.039724  0.020938   
Bottleneck                            -0.281138  0.030981 -0.193490  0.016268   
SR Overlap                            -0.034477  0.028188  0.022261  0.015745   
Opt Dist                              -0.321034  0.032706  0.003721  0.020555   
Goal Dist                             -0.302836  0.044120 -0.062585  0.022812   
Start Dist                            -0.388354  0.042585  0.023432  0.022548   
Wall Dist                             -0.489086  0.216298 -0.860005  0.120300   
Center Dist                            0.794073  0.219937  1.075171  0.122291   

            Process-Tracing Exp (Critical Mazes)                                
                                        Hovering            Duration            
                                         $\beta$        SE   $\beta$        SE  
Intercept                               0.429356  0.302512  5.827896  0.190236  
VGC                                     0.812465  0.064484  0.610643  0.038744  
Traj HS                                -0.162301  0.145443  0.080605  0.081969  
Graph HS                                0.684372  0.139669 -0.053737  0.084054  
Bottleneck                              0.001805  0.063510 -0.092728  0.032073  
SR Overlap                              0.535115  0.252113 -0.122904  0.122494  
Opt Dist                                0.103299  0.049630  0.000546  0.027976  
Goal Dist                              -0.471198  0.321483  0.478437  0.194107  
Start Dist                             -0.634401  0.298251  0.470346  0.185898  
Wall Dist                               0.552063  0.278745  0.354177  0.171129  
Center Dist                             0.316360  0.236698  0.000123  0.161954

## Create lesioned table

In [7]:
all_lesioning_res = [
    dict(res=exp1_lesioning_res, name=("Initial Exp", "Awareness")),
    dict(res=exp2_full_lesioning_res, name=("Up-front Exp", "Awareness (F)")),
    dict(res=exp2_early_lesioning_res, name=("Up-front Exp","Awareness (E)")),
    dict(res=exp3_correct_lesioning_res, name=("Critical Mazes Exp", "Accuracy")),
    dict(res=exp3_conf_lesioning_res, name=("Critical Mazes Exp", "Confidence")),
    dict(res=exp3_attn_lesioning_res, name=("Critical Mazes Exp", "Awareness")),
    dict(res=exp4a_hov_lesioning_res, name=("Process-Tracing Exp (Initial Mazes)", "Hovering")),
    dict(res=exp4a_dur_lesioning_res, name=("Process-Tracing Exp (Initial Mazes)", "Duration")),
    dict(res=exp4b_hov_lesioning_res, name=("Process-Tracing Exp (Critical Mazes)", "Hovering")),
    dict(res=exp4b_dur_lesioning_res, name=("Process-Tracing Exp (Critical Mazes)", "Duration")),
]

### Likelihood ratio test summary

In [8]:
all_summaries = []
for mod in all_lesioning_res:
    all_summaries.append(
        f"{', '.join(mod['name'])}: {mod['res'].likelihood_ratio_test('vgc_weight_Z', 1)}"
    )
all_summaries = '; \n'.join(all_summaries)
with open("./inputs/all_plan_exp_vgc_global_lesion_llr.tex", "w") as file:
    file.write(all_summaries)
print(all_summaries)

Initial Exp, Awareness: $\chi^2(1) = 400.63, < 1.0 \times 10^{-16}$; 
Up-front Exp, Awareness (F): $\chi^2(1) = 203.67, < 1.0 \times 10^{-16}$; 
Up-front Exp, Awareness (E): $\chi^2(1) = 127.79, < 1.0 \times 10^{-16}$; 
Critical Mazes Exp, Accuracy: $\chi^2(1) = 86.07, < 1.0 \times 10^{-16}$; 
Critical Mazes Exp, Confidence: $\chi^2(1) = 173.12, < 1.0 \times 10^{-16}$; 
Critical Mazes Exp, Awareness: $\chi^2(1) = 356.61, < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Initial Mazes), Hovering: $\chi^2(1) = 289.13, < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Initial Mazes), Duration: $\chi^2(1) = 89.71, < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Critical Mazes), Hovering: $\chi^2(1) = 210.47, < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Critical Mazes), Duration: $\chi^2(1) = 242.78, < 1.0 \times 10^{-16}$


In [9]:
def format_global_lesioning_res(res):
    return \
        res[['dAIC.FULL', 'lesioned_effect']].\
        set_index('lesioned_effect')["dAIC.FULL"].drop('FULL')

global_lesion_table = pd.concat(
    [format_global_lesioning_res(res['res']) for res in all_lesioning_res],
    axis=1,
    keys=[res['name'] for res in all_lesioning_res]
)


global_lesion_table = global_lesion_table.reindex(predictors)
global_lesion_table.index = ["Intercept" if "Intercept" in i else short_predictor_names[i.replace("_Z", "")] for i in global_lesion_table.index.values]

# with annotation
col_ranks = global_lesion_table.rank(axis=0, ascending=False)
annot_global_lesion_table = []
for model, row in global_lesion_table.iterrows():
    annot_row = []
    for measure, daic in row.to_dict().items():
        if np.isnan(daic):
            annot_row.append("---")
            continue
            
        daic_rank = col_ranks.loc[model, measure]
        if daic_rank == 1:
            annot = "$^*$"
        elif daic_rank == 2:
            annot = r"\textsuperscript{\textdagger}"
        else:
            annot = ""
        annot_row.append(str(int(daic))+annot)
    annot_global_lesion_table.append(annot_row)

annot_global_lesion_table = pd.DataFrame(annot_global_lesion_table, index=global_lesion_table.index, columns=global_lesion_table.columns)
global_lesion_string = annot_global_lesion_table.to_latex(
    column_format="lcccccccccc",
    col_space=1,
    escape=False,
    # caption=
)

# without annotation
# global_lesion_string = global_lesion_table.round(0).to_latex(
#     na_rep="---", 
#     float_format=lambda f: str(int(f)) if not np.isnan(f) else "---",
#     column_format="lcccccccccc",
#     col_space=1,
#     escape=False
# )


global_lesion_string = global_lesion_string.replace(
    r"{} &                       Initial Exp & \multicolumn{2}{l}{Up-front Exp} & \multicolumn{3}{l}{Critical Mazes Exp} & \multicolumn{2}{l}{Process-Tracing Exp (Initial Mazes)} & \multicolumn{2}{l}{Process-Tracing Exp (Critical Mazes)} \\",
    r"{} & Initial Exp & \multicolumn{2}{c}{Up-front Exp} & \multicolumn{3}{c}{Critical Mazes Exp} & \multicolumn{2}{>{\centering}m{2.5cm}}{Process-Tracing Exp (Initial Mazes)} & \multicolumn{2}{>{\centering}m{2.5cm}}{Process-Tracing Exp (Critical Mazes)} \\" + \
    "\n" + r"\cmidrule(lr){2-2}\cmidrule(lr){3-4}\cmidrule(lr){5-7}\cmidrule(lr){8-9}\cmidrule(lr){10-11}",
)

global_lesion_string = global_lesion_string.replace(
    r"\bottomrule",
    r"\bottomrule"+"\n"+r"\multicolumn{10}{l}{$^*$Largest increase in AIC after lesioning; \textsuperscript{\textdagger}Second-largest increase}"
)

with open("results/global_lesion_table.tex", "w") as file:
    file.write(
        textwrap.dedent(r"""
        \documentclass{standalone}
        \usepackage{booktabs}
        \usepackage{tabularx}
        \usepackage{caption}
        \usepackage[scaled=0.7025996185887785]{helvet}
        \linespread{0.7025996185887785}
        \renewcommand{\familydefault}{\sfdefault}
        \begin{document}
        \fontsize{7}{8.4}
        """) + \
        global_lesion_string + \
        textwrap.dedent(r"""
        \end{document}
        """)
    )

# Create pdf to combine in vector image editor
# need to figure out how to make text in right font and also recognizable as text in eps
process = subprocess.Popen("xelatex -output-directory results results/global_lesion_table.tex".split())
output, error = process.communicate()
print(output, error)
global_lesion_table

None None


Initial Exp  Up-front Exp               Critical Mazes Exp  \
                Awareness Awareness (F) Awareness (E)           Accuracy   
VGC            398.629279    201.667360    125.790396          84.071725   
Traj HS         57.647201     23.555357      5.485984           5.348129   
Graph HS       235.913805     65.882923     40.932330           2.736735   
Bottleneck      91.170194     92.994807    118.648818          -1.639711   
SR Overlap     130.761984     73.956184     67.638443           7.006124   
Nav Dist       892.558356    434.429980           NaN          23.491889   
Nav Dist Step   37.507607     44.742686           NaN          -1.211688   
Opt Dist              NaN           NaN    186.258062                NaN   
Goal Dist       23.108310      7.024620     13.260446           4.124637   
Start Dist      33.862169      0.024784     12.578553           7.757866   
Wall Dist       49.887254      4.013621     13.496810           1.700364   
Center Dist     46.077476      7.076624     16.803076           5.647953   

                                      Process-Tracing Exp (Initial Mazes)  \
               Confidence   Awareness                            Hovering   
VGC            171.121804  354.606839                          287.132794   
Traj HS         23.307258   48.983031                           12.774103   
Graph HS         3.411671   24.292480                           -1.086140   
Bottleneck       4.211653   45.454788                           81.272689   
SR Overlap       6.130127    8.427774                           -0.508374   
Nav Dist        16.065125  170.433964                                 NaN   
Nav Dist Step   -0.880638   45.765821                                 NaN   
Opt Dist              NaN         NaN                           95.597504   
Goal Dist       -1.271890   13.325886                           45.481815   
Start Dist      -1.847150   18.095001                           82.342763   
Wall Dist        1.246944   17.503439                            3.049318   
Center Dist      1.012345   25.908090                           10.807250   

                          Process-Tracing Exp (Critical Mazes)              
                 Duration                             Hovering    Duration  
VGC             87.710989                           208.466715  240.780766  
Traj HS         -0.428221                            -1.294494   -1.224681  
Graph HS         1.598696                            24.604416   -1.631500  
Bottleneck     137.465621                             0.767702    4.471245  
SR Overlap      -0.001321                            28.010180   -1.045624  
Nav Dist              NaN                                  NaN         NaN  
Nav Dist Step         NaN                                  NaN         NaN  
Opt Dist        -1.967529                            -1.220997   -1.999628  
Goal Dist        5.385377                             0.457686    3.552924  
Start Dist      -0.944068                             4.694526    3.602572  
Wall Dist       48.910659                            15.353723    2.149589  
Center Dist     74.881809                            -1.920957   -1.999999

In [10]:
!xelatex -output-directory results results/global_lesion_table.tex

This is XeTeX, Version 3.14159265-2.6-0.99992 (TeX Live 2015) (preloaded format=xelatex)
 restricted \write18 enabled.
entering extended mode
(./results/global_lesion_table.tex
LaTeX2e <2015/01/01>
Babel <3.9l> and hyphenation patterns for 79 languages loaded.
(/usr/local/texlive/2015/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2012/09/15 v1.1b Class to compile TeX sub-files stan
dalone
(/usr/local/texlive/2015/texmf-dist/tex/generic/oberdiek/ifluatex.sty)
(/usr/local/texlive/2015/texmf-dist/tex/generic/oberdiek/ifpdf.sty)
(/usr/local/texlive/2015/texmf-dist/tex/generic/ifxetex/ifxetex.sty)
(/usr/local/texlive/2015/texmf-dist/tex/latex/xkeyval/xkeyval.sty
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/xkeyval.tex
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/xkvutils.tex
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/keyval.tex))))
(/usr/local/texlive/2015/texmf-dist/tex/latex/standalone/standalone.cfg)
(/usr/local/texlive/2015/

In [11]:
annot_global_lesion_table.to_latex()

'\\begin{tabular}{lllllllllll}\n\\toprule\n{} &                       Initial Exp & \\multicolumn{2}{l}{Up-front Exp} & \\multicolumn{3}{l}{Critical Mazes Exp} & \\multicolumn{2}{l}{Process-Tracing Exp (Initial Mazes)} & \\multicolumn{2}{l}{Process-Tracing Exp (Critical Mazes)} \\\\\n{} &                         Awareness &                     Awareness (F) &                     Awareness (E) &                         Accuracy &                       Confidence &                         Awareness &                            Hovering &                         Duration &                             Hovering &                        Duration \\\\\n\\midrule\nVGC           &  398\\textbackslash textsuperscript\\{\\textbackslash textdagger\\} &  201\\textbackslash textsuperscript\\{\\textbackslash textdagger\\} &  125\\textbackslash textsuperscript\\{\\textbackslash textdagger\\} &                           84\\$\\textasciicircum *\\$ &                          171\\$\\textasciicircum *\\$